In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models

label_classes =['ants', 'aphids', 'armyworm', 'bees', 'beetle', 'bollworm', 'catterpillar', 'earthworms', 'earwig', 'grasshopper', 'mites', 'mosquito', 'moth', 'sawfly', 'slug', 'snail', 'stem_borer', 'wasp', 'weevil']

num_classes = len(label_classes)
loaded_model = models.densenet121(pretrained=False)  
loaded_model.classifier = nn.Linear(loaded_model.classifier.in_features, num_classes)  
loaded_model.load_state_dict(torch.load('cnn_model.pth'))
device = 'cpu'
loaded_model = loaded_model.to(device)
loaded_model.eval()

from torchvision import transforms
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

image_path = input("Enter The path:")
image = Image.open(image_path)

input_image = transform(image).unsqueeze(0).to(device)  

with torch.no_grad():
    output = loaded_model(input_image)
    _, predicted_class = torch.max(output, 1)

predicted_label = label_classes[predicted_class.item()]

print("Predicted label:", predicted_label)
#pest_insect_dataset/train/ants/ants (10).jpg

C:\Users\HARISH\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HARISH\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Enter The path: pest_insect_dataset/test/catterpillar/catterpillar (18).jpg


Predicted label: catterpillar


In [2]:
import requests

def search_pesticides(pest_name):
    try:
        url = "https://en.wikipedia.org/w/api.php"
        params = {
            "action": "query",
            "format": "json",
            "list": "search",
            "srsearch": f"about insect {pest_name} and pesticide to kill",
            "srprop": "snippet",
            "srlimit": "5"  
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        # Extract search results
        search_results = data['query']['search']
        return search_results
    except requests.exceptions.RequestException as e:
        print("Error retrieving data:", e)
        return None

def display_search_results(search_results):
    if not search_results:
        print("No search results found.")
        return
    print("About and Pesticide :")
    for result in search_results:
        title = result['title']
        snippet = result['snippet'].replace('<span class="searchmatch">', '').replace('</span>', '')
        print(f"-> {title.upper()}:")
        print(f"  -{snippet}")

pest_name = predicted_label.strip()
print(pest_name.upper()+str(":"))
if not pest_name:
    print("Please enter a valid pest name.")
search_results = search_pesticides(pest_name)
display_search_results(search_results)


CATTERPILLAR:
No search results found.
